In [21]:
! git clone https://github.com/Xiar-fatah/Stock-Market-Patterns

fatal: destination path 'Stock-Market-Patterns' already exists and is not an empty directory.


In [2]:
# Importing necessary tools for machine learning
import torch
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go


In [4]:
# Importing data.py and ERRORS.py
import sys
sys.path.append('/content/Stock-Market-Patterns/Core')
import data
import ERRORS

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size,batch_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size 
        self.num_layers = num_layers 
        self.output_size = output_size 
        self.batch_size = batch_size 
        self.hidden_state = self.hidden_cell()
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, dropout = 0.5)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def hidden_cell(self):
        h_0,c_0 = (torch.zeros(self.num_layers, self.batch_size, self.hidden_size), 
        torch.zeros(self.num_layers, self.batch_size, self.hidden_size))
        return h_0,c_0 
    
    def forward(self, t):
        t = t.reshape(20,1,1)
        t, self.hidden_state = self.lstm(t, (self.hidden_state))
        t = self.linear(self.hidden_state[0][-1].detach())
        return t[0]

In [6]:
model = LSTM(input_size = 1, hidden_size = 512,
                num_layers = 1, output_size = 1, batch_size = 1)
learning_rate = 0.0005
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 1000
# For plot title
hidden_size = 512
num_layers = 1


C:\Users\tariq\.conda\envs\pytorch\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1



In [ ]:
train = data.trainloader
test = data.testloader

for epoch in range(epochs):
    for i, (seq, labels) in enumerate(train):
        optimizer.zero_grad()
        output = model(seq)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        
    if (epoch + 1) % 10 == 0:
            print("Epoch: {}, Loss: {:.5f}".format(epoch + 1, loss.item()))

    #Evaluation
    model.eval()
    predictions = []
    with torch.no_grad():
        for i, (seq, labels) in enumerate(test):
            output = model(seq)
            predictions.append(output.tolist()[0])
    predictions = np.add(np.multiply(predictions,data.std),data.mean)
    real = data.real
    print("RMS: " + str(ERRORS.RMS(predictions, real)) + "\n"
        "MAPE: " + str(ERRORS.MAPE(predictions, real)) + "\n"
        "MAE: " + str(ERRORS.MAE(predictions, real)) + "\n"
        "R: " + str(ERRORS.R(predictions, real)))    



    x = np.linspace(0,999,1000)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
                    x = x,
                    y = real,
                    name="Closing price",
                    line_color='deepskyblue',
                    opacity=0.8))

    fig.add_trace(go.Scatter(
                    x = x,
                    y = predictions,
                    name="Predictions",
                    line_color='dimgray',
                    opacity=0.8))
    fig.update_layout(title_text='Plot for learning rate: ' + str(learning_rate) + ' , epoch: ' + str(epoch + 1) + ' hidden_size: ' + str(hidden_size) + ' and num_layers: ' + str(num_layers),
                      xaxis_rangeslider_visible=True,
                      xaxis_title='Time',
                      yaxis_title='US Dollars')

    fig.show()

RMS: 1.2388202867606768
MAPE: 0.013708952577974928
MAE: 0.14255688978255618
R: 0.9998740020160959
